# Statistical Inference of Luminosity and Temperature of Dwarf Stars

### Ahmed Rizk, Cathy Liu, Divya Bilolikar, Olivia Pang

In [8]:
install.packages("infer")
library(tidyverse)
library(infer)
library(gridExtra)
library(testthat)
library(digest)
library(broom)

Installing package into ‘/home/jupyter/R/x86_64-pc-linux-gnu-library/4.1’
(as ‘lib’ is unspecified)

also installing the dependency ‘patchwork’



Attaching package: ‘gridExtra’


The following object is masked from ‘package:dplyr’:

    combine



Attaching package: ‘testthat’


The following object is masked from ‘package:dplyr’:

    matches


The following object is masked from ‘package:purrr’:

    is_null


The following objects are masked from ‘package:readr’:

    edition_get, local_edition


The following object is masked from ‘package:tidyr’:

    matches




## Introduction

In this project we want to determine if there is a difference in luminosity and temperature between red, brown, and white dwarfs. Our location parameter is the mean of luminosity and temperature and our scale parameter is the standard deviation of both luminosity and temperature. We chose these because our data is approximately normally distributed and its not skewed particularly.

Stars are typically classified into four different groups – white dwarfs, main sequence, giants, and supergiants (Chiosi). A typical star will become a main sequence star and then end its lifecycle as a white dwarf, these are the most common stars (“Main Sequence Stars: Definition & Life Cycle.” ). Red dwarfs are not a type of white dwarf, rather they are a type of small main sequence star that is not very bright. Another type of dwarf is a brown dwarf. These are not usually classified as stars because they do not burn hot enough (“Red Dwarfs: The Most Common and Longest-Lived Stars.”). White dwarfs are very small dense stars that are not very bright when compared to main sequence stars (Koester). When a massive star is formed it will become a giant star or a supergiant and then end its lifecycle as a neutron star or a blackhole. Giants and supergiants are very large stars (with supergiant being even larger) that shine very bright when compared to main sequence stars (Chiosi). 

The dataset that are using has the following variables

| Variable | Description |
| :-: | :-: |
| Absolute Temperature (K) | The temperature of the star in Kelvin |
| Relative Luminosity (L/Lo) | The luminosity of the star when divided by the average luminosity of the sun (3.828 x 10^26 Watts) |
| Relative Radius (R/Ro) | The radius of the star when divided by the average radius of the sun (6.9551 x 10^8 m) |
| Absolute Magnitude (Mv) | The absolute magnitude of the star |
| Star Color | The color of the star |
| Spectral Class (O,B,A,F,G,K,M) | If the star is a main sequence star, the type of main sequence star it is |
| Star Type (Red Dwarf, Brown Dwarf, White Dwarf, Main Sequence , SuperGiants, HyperGiants) | The type of star it is | 

## Preliminary Analysis

### Reading the dataset into R

In [ ]:
stars <- read.csv("6-class.csv")

glimpse(stars)

### Cleaning and Wrangling

In [ ]:
# change 'Star.type' to type 'factor' so that it can be treated as a categorical variable

stars <- mutate(stars, Star.type = as_factor(Star.type))

In [ ]:
# Filtering out the 'Giant' stars 

dwarfs <- stars %>%
filter(Star.type != 5) %>%
filter(Star.type != 4) %>%
filter(Star.type != 3)


### Visualizations

In [ ]:
# Creating boxplots to show the variation in luminosity and temperature across star types

L_box <- ggplot(dwarfs, aes(x = Star.type, y = Luminosity.L.Lo.)) + geom_boxplot() + ggtitle("Luminosity vs Star Type") + xlab("Star Type") + ylab("Luminosity (Lo)")
T_box <- ggplot(dwarfs, aes(x = Star.type, y = Temperature..K.)) + geom_boxplot() +  ggtitle("Temperature vs Star Type") +  xlab("Star Type") + ylab("Temperature (K)")
grid.arrange(L_box, T_box, ncol=2)

In the figure above we can see the distributions of luminosity vs star type and temperature vs star type. We can see that the means of the luminosity across star types is very similar however the interquartile range of star type 1 is much larger. In the temperature vs star type graph we can see that the range and mean of star type  0 and 1 are very similar however both the mean and interquartile range of star type 2 is very large.

In [ ]:
# Isolate each dwarf star type

type_0 <- filter(stars, Star.type == 0)
type_1 <- filter(stars, Star.type == 1)
type_2 <- filter(stars, Star.type == 2)

#### Brown Dwarf Distributions

In [ ]:
# Find the distribution of luminosity and temperature values for the brown dwarf star

L_0 <- ggplot(type_0, aes(x = Luminosity.L.Lo.)) + geom_histogram(bins = 20)
T_0 <- ggplot(type_0, aes(x = Temperature..K.)) + geom_histogram(bins = 20)
grid.arrange(L_0, T_0, ncol=2)

In the Luminosity graph above we can see that the distribution is unimodal and looks fairly normal with a few larger outlier. Most stars seem to have a luminosity between 0 and 0.001 LL_o. In the Temperature graph we can see that the dsitibution looks bimodal peaking at around 2800 K and 3500 K.

#### Red Dwarf Distributions

In [ ]:
# Find the distribution of luminosity and temperature values for the red dwarf star

L_1 <- ggplot(type_1, aes(x = Luminosity.L.Lo.)) + geom_histogram(bins = 20)
T_1 <- ggplot(type_1, aes(x = Temperature..K.)) + geom_histogram(bins = 20)
grid.arrange(L_1, T_1, ncol=2)

In the Luminosity graph above we can see that the distribution is unimodal and looks fairly normal with a few larger outlier. Most stars seem to have a luminosity between 0 and 0.01 LL_o. In the Temperature graph we can see that the distribution looks unimodal peaking in the range of 3100K to 3500K.

#### White Dwarf Distributions

In [ ]:
# Find the distribution of luminosity and temperature values for the white dwarf star

L_2 <- ggplot(type_2, aes(x = Luminosity.L.Lo.)) + geom_histogram(bins = 20)
T_2 <- ggplot(type_2, aes(x = Temperature..K.)) + geom_histogram(bins = 20)
grid.arrange(L_2, T_2, ncol=2)

In the Luminosity graph above we can see that the distribution is unimodal and looks fairly normal with a few larger outlier. Most stars seem to have a luminosity between 0 and 0.005 LL_o. In the Temperature graph we can see that the distribution looks right skewed uni modal.

### Computing Parameter Estimates

#### Brown Star Estimates

In [ ]:
# Finding the mean and standard deviation of the luminosity and temperature of the brown star

brown_dwarf_stats <- type_0 %>% 
    summarise(mean_lum = mean(Luminosity.L.Lo.), 
              sd_lum = sd(Luminosity.L.Lo.),
              mean_temp = mean(Temperature..K.),
              sd_temp = sd(Temperature..K.)) %>%
    add_column(Type = "Brown", .before = "mean_lum")

brown_dwarf_stats

In [ ]:
# Finding the mean and standard deviation of the luminosity and temperature of the red star

red_dwarf_stats <- type_1 %>% 
    summarise(mean_lum = mean(Luminosity.L.Lo.), 
              sd_lum = sd(Luminosity.L.Lo.),
              mean_temp = mean(Temperature..K.),
              sd_temp = sd(Temperature..K.)) %>%
    add_column(Type = "Red", .before = "mean_lum")

red_dwarf_stats

In [ ]:
# Finding the mean and standard deviation of the luminosity and temperature of the white star

white_dwarf_stats <- type_2 %>% 
    summarise(mean_lum = mean(Luminosity.L.Lo.), 
              sd_lum = sd(Luminosity.L.Lo.),
              mean_temp = mean(Temperature..K.),
              sd_temp = sd(Temperature..K.)) %>%
    add_column(Type = "White", .before = "mean_lum")

white_dwarf_stats

In [ ]:
# Combining all parameter estimates together

parameter_estimates <- rbind(brown_dwarf_stats, red_dwarf_stats, white_dwarf_stats)
parameter_estimates

In [5]:
#bootstrap
brown_dwarf_stats <- type_0 %>% 
    rep_sample_n(size = 40, reps = 1000, replace = TRUE) %>% 
    group_by(replicate) %>% 
    summarise(mean_lum = mean(Luminosity.L.Lo.), 
              mean_temp = mean(Temperature..K.),
              sd_lum = sd(Luminosity.L.Lo.),
              sd_temp = sd(Temperature..K.)) %>%
    summarise(mean_lum = mean(mean_lum), 
              mean_temp = mean(mean_temp), 
              SE_lum = mean(sd_lum), 
              SE_temp = mean(sd_temp)) %>%
    add_column(Type = "Brown", .before = "mean_lum")

red_dwarf_stats <- type_1 %>% 
    rep_sample_n(size = 40, reps = 1000, replace = TRUE) %>% 
    group_by(replicate) %>% 
    summarise(mean_lum = mean(Luminosity.L.Lo.), 
              mean_temp = mean(Temperature..K.),
              sd_lum = sd(Luminosity.L.Lo.),
              sd_temp = sd(Temperature..K.)) %>%
    summarise(mean_lum = mean(mean_lum), 
              mean_temp = mean(mean_temp), 
              SE_lum = mean(sd_lum), 
              SE_temp = mean(sd_temp)) %>%
    add_column(Type = "Red", .before = "mean_lum")

white_dwarf_stats <- type_2 %>% 
    rep_sample_n(size = 40, reps = 1000, replace = TRUE) %>% 
    group_by(replicate) %>% 
    summarise(mean_lum = mean(Luminosity.L.Lo.), 
              mean_temp = mean(Temperature..K.),
              sd_lum = sd(Luminosity.L.Lo.),
              sd_temp = sd(Temperature..K.)) %>%
    summarise(mean_lum = mean(mean_lum), 
              mean_temp = mean(mean_temp), 
              SE_lum = mean(sd_lum), 
              SE_temp = mean(sd_temp)) %>%
    add_column(Type = "White", .before = "mean_lum")

bootstrap_estimates <- rbind(brown_dwarf_stats, red_dwarf_stats, white_dwarf_stats)
bootstrap_estimates

ERROR: Error in rep_sample_n(., size = 40, reps = 1000, replace = TRUE): could not find function "rep_sample_n"


## Methods: Plan

This report is trustworthy because the dataset that we use is fairly large, so it is potentially of the true population. Therefore the mean estimates that we have calculated can be good starting points to create confidence intervals for our population parameters. While our estimations are a good starting point, they are not enough to present to a stakeholder. The plots would be a better representation if we were to do some further data processing like bootstrapping or asymptotic testing. Additionally there is no way to estimate the sampling variability of the population of stars by examining point estimates of the standard deviation, or the distribution of values of the dataset. 

In order to address the gap we brought up above, we need to bootstrap to more closely approximate the actual distribution of the population, and by extension find a range of possible parameter estimates as opposed to a point estimate.
In the final report, we expect to find whether we are confident to say there is a difference in the luminosity and temperature between dwarf stars. We will express our result and the uncertainty in a trustworthy way. We believe finding this clarifies the difference in some features between different types of dwarf stars, and further increases the accuracy and reliability of star classification. This analysis leads to future questions such as “is there a difference in the magnitude between dwarf stars”. Additionally we can use these results to infer real astronomical observations.



# ANOVA

ANOVA (Analysis of Variance) is a test to determine whether the difference between means across three or more groups exist by analyzing the overall and within-group variance. ANOVA test calculates F-value, which is the ratio of the variability between group means and the variation within groups, and corresponding p-value. If F-value is significantly greater than 1, the p-value is going to be small, we conclude that the variability between group means exists. In this project, since our goal is to analyze the difference between temperature and luminosity of three groups of dwarf planets, ANOVA becomes very appropriate and effective. In this section, we will conduct two ANOVA tests across three dwarf groups on both variables, and analyze the results.


### ANOVA for Temperature Across all Three Dwarf Types

We start with temperature. Before actually starting the ANOVA test, we need to do a little bit of cleanup, select the columns that we want, which are Star Type and Temperature.

In [ ]:
# filter out the target column Temperature to get the tidy data set
dwarfs_temp <- dwarfs %>%
               select(Temperature..K., Star.type)
head(dwarfs_temp)

In [ ]:
# Perform ANOVA test and print the result table
anova_temp <- aov(Temperature..K.~Star.type, data = dwarfs_temp)%>%
              tidy()
print(anova_temp)

In [ ]:
# Filter out the F-value from the table above
# Filter out the p-value from the table above
f_stat_temp <- anova_temp %>%
          filter(term == "Star.type") %>%
          select(statistic) %>%
          as.numeric()
anova_pval_temp <- anova_temp %>%
              filter(term == "Star.type") %>%
              select(p.value) %>%
              as.numeric()
print(f_stat_temp)
print(anova_pval_temp)

The F-value obtained is 188.91, and the p-value is 2.74e-37. In this case, our F-value is significantly greater than 1. Therefore, the variation between group means is clearly larger than the variation within groups, indicating that at least one of the mean value is different across three groups. The p-value also states the result. Since p-value is significantly smaller than the critical value (0.05), it falls in reject region. Therefore, by looking at the ANOVA test for temperature across three groups, we say at least one of the mean value is different across three groups and the temperature of three dwarf groups are not all same.

### ANOVA for Luminosity Across all Three Dwarf Types

In [ ]:
# filter out the target column Luminosity to get the tidy data set
dwarfs_lumi <- dwarfs %>%
               select(Luminosity.L.Lo., Star.type)
head(dwarfs_lumi)

In the following two blocks of code, we perform ANOVA test on luminosity across three groups of dwarfs.

In [ ]:
# Perform ANOVA test and print the result table
anova_lumi <- aov(Luminosity.L.Lo.~Star.type, data = dwarfs_lumi)%>%
              tidy()
print(anova_lumi)

In [ ]:
# Filter out the F-value from the table above
# Filter out the p-value from the table above
f_stat_lumi <- anova_lumi %>%
          filter(term == "Star.type") %>%
          select(statistic) %>%
          as.numeric()
anova_pval_lumi <- anova_lumi%>%
              filter(term == "Star.type") %>%
              select(p.value) %>%
              as.numeric()
print(f_stat_lumi)
print(anova_pval_lumi)

The F-value obtained is 5.09, and the p-value is 0.0076. Although we get a much smaller F-value than the F-value in temperature test, it still clearly larger than 1. The p-value is also smaller than the critical value (0.05), and falls in reject region. Although we might say the difference between luminosity across groups is not as significantly as the difference between temperature, we are still confident to say at least one mean value is different from others, and the luminosity of three dwarf groups are not all same.

## Citation

Chiosi, Cesare, Gianpaolo Bertelli, and Alessandro Bressan. "New developments in understanding the HR diagram." Annual review of astronomy and astrophysics 30.1 (1992): 235-285.

Koester, Detlev, and Ganesar Chanmugam. "Physics of white dwarf stars." Reports on Progress in Physics 53.7 (1990): 837.

Tillman, Nola Taylor, and Ben Biggs. “Main Sequence Stars: Definition & Life Cycle.” Space.com, Space, 26 Jan. 2022, https://www.space.com/22437-main-sequence-star.html. 

Tillman, Nola Taylor. “Red Dwarfs: The Most Common and Longest-Lived Stars.” Space.com, Space, 6 June 2019, https://www.space.com/23772-red-dwarf-stars.html. 